In [1]:
import pathlib
import shutil
import sys

import numpy as np
import tqdm

sys.path.append(str(pathlib.Path("../../utils").resolve()))
from file_checking import check_number_of_files

In [2]:
overwrite = True

In [3]:
# set path to the processed data dir
processed_data_dir = pathlib.Path("../processed_data").resolve(strict=True)
normalized_data_dir = pathlib.Path("../../data/NF0014/normalized_z").resolve(
    strict=True
)
# normalized_data_dir = pathlib.Path("../../data/test_dir").resolve(strict=True)
cellprofiler_dir = pathlib.Path("../../data/cellprofiler").resolve()
if cellprofiler_dir.exists():
    shutil.rmtree(cellprofiler_dir)
else:
    cellprofiler_dir.mkdir(parents=True, exist_ok=True)

In [4]:
# perform checks for each directory
processed_data_dir_directories = list(processed_data_dir.glob("*"))
normalized_data_dir_directories = list(normalized_data_dir.glob("*"))
cellprofiler_dir_directories = list(cellprofiler_dir.glob("*"))

print(
    f"""
      #################################################################################\n
      ## Checking the number of files in each subdirectory of:\n 
      ## {processed_data_dir.absolute()}\n
      #################################################################################
      """
)
for file in processed_data_dir_directories:
    check_number_of_files(file, 6)


print(
    f"""
      #################################################################################\n
      ## Checking the number of files in each subdirectory of:\n 
      ## {normalized_data_dir.absolute()}\n
      #################################################################################
      """
)
for file in normalized_data_dir_directories:
    check_number_of_files(file, 5)


      #################################################################################

      ## Checking the number of files in each subdirectory of:
 
      ## /home/lippincm/Documents/GFF_3D_organoid_profiling_pipeline/1.segment_images/processed_data

      #################################################################################
      
C11-2 expected 6 files, but found 4 files.
C4-2 expected 6 files, but found 7 files.
C2-1 expected 6 files, but found 8 files.

      #################################################################################

      ## Checking the number of files in each subdirectory of:
 
      ## /home/lippincm/Documents/GFF_3D_organoid_profiling_pipeline/data/NF0014/normalized_z

      #################################################################################
      


## Copy the normalized images to the cellprofiler images dir

In [5]:
# get the list of dirs in the normalized_data_dir
norm_dirs = [x for x in normalized_data_dir.iterdir() if x.is_dir()]
# copy each dir and files to cellprofiler_dir
for norm_dir in tqdm.tqdm(norm_dirs):
    dest_dir = pathlib.Path(cellprofiler_dir, norm_dir.name)
    if dest_dir.exists() and overwrite:
        shutil.rmtree(dest_dir)
        shutil.copytree(norm_dir, dest_dir)
    elif not dest_dir.exists():
        shutil.copytree(norm_dir, dest_dir)
    else:
        pass

100%|██████████| 104/104 [01:32<00:00,  1.12it/s]


## Copy files from processed dir to cellprofiler images dir

In [6]:
# get a list of dirs in processed_data
dirs = [x for x in processed_data_dir.iterdir() if x.is_dir()]
file_extensions = {".tif", ".tiff"}
# get a list of files in each dir
for well_dir in tqdm.tqdm(dirs):
    files = [x for x in well_dir.iterdir() if x.is_file()]
    for file in files:
        if file.suffix in file_extensions:
            # copy each of the raw files to the cellprofiler_dir for feature extraction
            new_file_dir = pathlib.Path(
                cellprofiler_dir, well_dir.name, file.stem + file.suffix
            )
            shutil.copy(file, new_file_dir)

100%|██████████| 6/6 [00:02<00:00,  2.40it/s]


In [7]:
jobs_to_rerun_path = pathlib.Path("../rerun_jobs.txt").resolve()
if jobs_to_rerun_path.exists():
    jobs_to_rerun_path.unlink()

In [8]:
dirs_in_cellprofiler_dir = [x for x in cellprofiler_dir.iterdir() if x.is_dir()]
dirs_in_cellprofiler_dir = sorted(dirs_in_cellprofiler_dir)
for dir in tqdm.tqdm(dirs_in_cellprofiler_dir):
    if not check_number_of_files(dir, 9):
        with open(jobs_to_rerun_path, "a") as f:
            f.write(f"{dir.name}\n")

100%|██████████| 104/104 [00:00<00:00, 9300.41it/s]

C11-2 expected 9 files, but found 7 files.
C2-1 expected 9 files, but found 10 files.
C2-2 expected 9 files, but found 5 files.
C3-1 expected 9 files, but found 5 files.
C3-2 expected 9 files, but found 5 files.
C4-1 expected 9 files, but found 5 files.
C5-1 expected 9 files, but found 5 files.
C5-2 expected 9 files, but found 5 files.
C6-1 expected 9 files, but found 5 files.
C6-2 expected 9 files, but found 5 files.
C7-1 expected 9 files, but found 5 files.
C7-2 expected 9 files, but found 5 files.
C8-1 expected 9 files, but found 5 files.
C8-2 expected 9 files, but found 5 files.
C9-1 expected 9 files, but found 5 files.
C9-2 expected 9 files, but found 5 files.
D10-1 expected 9 files, but found 5 files.
D10-2 expected 9 files, but found 5 files.
D11-1 expected 9 files, but found 5 files.
D11-2 expected 9 files, but found 5 files.
D11-3 expected 9 files, but found 5 files.
D2-1 expected 9 files, but found 5 files.
D2-2 expected 9 files, but found 5 files.
D2-3 expected 9 files, but 

In [9]:
# move an example to the example dir
example_dir = pathlib.Path("../processed_data/C4-2/gifs/").resolve(strict=True)
final_example_dir = pathlib.Path("../examples/segmentation_output/C4-2/gifs").resolve()
if final_example_dir.exists():
    shutil.rmtree(final_example_dir)


if example_dir.exists():
    shutil.copytree(example_dir, final_example_dir)